<a href="https://colab.research.google.com/github/callumselv/Y4_project/blob/main/Lohani_2021_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qutip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 38.4 MB/s eta 0:00:00


In [2]:
import numpy as np
from qutip import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from scipy.stats import unitary_group

In [3]:
no_data = 30000
d = 3
dimension = 2**d

In [ ]:
# Generate rho matrices

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

In [ ]:
# Generate rho matrices

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

In [ ]:
cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

In [4]:
def gen_rhos_chos(no_data, d):
    dimension = 2**d
    rhos = np.zeros((no_data,dimension,dimension),dtype=complex)

    eta = 1e-7

    # Generate random density matrices with eta adjustment to avoid issues in Cholesky decomposition
    for i in range(no_data):
        random_unitary = unitary_group.rvs(dimension)
        coeffs = random_unitary[:,0]
        rho = np.outer(coeffs,coeffs.conj())
        rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

    # Calculate Cholesky decomposition of density matrices
    cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

    return rhos,cholesky_decompositions

In [ ]:
for i in cholesky_decompositions[11000]:
    print(i)

[0.7367341+0.j 0.       +0.j 0.       +0.j 0.       +0.j]
[3.22038541e-01+0.36353001j 1.89377159e-04+0.j
 0.00000000e+00+0.j         0.00000000e+00+0.j        ]
[2.73129519e-01-1.21787911e-01j 1.06247803e-05-3.36880370e-05j
 1.66947350e-04+0.00000000e+00j 0.00000000e+00+0.00000000e+00j]
[ 1.19477978e-01-3.43005622e-01j -2.09691287e-05-3.74295840e-05j
  1.43099125e-05-1.52190244e-05j  1.69704006e-04+0.00000000e+00j]


In [ ]:
all_taus = np.zeros((no_data, dimension**2))
for no in range(no_data):
    for cholesky in cholesky_decompositions:
        extracted_values = []
        for i in range(cholesky.shape[0]):
            extracted_values.append(cholesky[i, i].real)

        primary_indices = [(1, 0), (2, 1), (3, 2)]
        for i, j in primary_indices:
            extracted_values.append(cholesky[i, j].real)
            extracted_values.append(cholesky[i, j].imag)

        secondary_indices = [(2, 0), (3, 1), (3, 0)]
        for i, j in secondary_indices:
            extracted_values.append(cholesky[i, j].real)
            extracted_values.append(cholesky[i, j].imag)
    all_taus[no] = np.array(extracted_values)


KeyboardInterrupt: 

In [ ]:
cholesky_decompositions.shape

(10000, 4, 4)

In [ ]:
indices = np.zeros()

primary_indices = [(i, i - 1) for i in range(1, d**2)]
secondary_indices = [(i, i - 2) for i in range(2, d**2)]
secondary_indices = [(i, i - 2) for i in range(2, d**2)]
print(primary_indices, secondary_indices)


[(1, 0), (2, 1), (3, 2)] [(2, 0), (3, 1)]


In [20]:
def generate_index_pairs(d):
    dimension = 2**d
    index_num = int(dimension*(dimension-1)/2)
    pairs = np.zeros((index_num,2), dtype=int)
    index=0
    for diff in range(1, dimension):
        for i in range(diff, dimension):
            j = i - diff
            pairs[index] = (i, j)
            index+=1

    return pairs

In [21]:
ind = generate_index_pairs(d)
ind

array([[1, 0],
       [2, 1],
       [3, 2],
       [4, 3],
       [5, 4],
       [6, 5],
       [7, 6],
       [2, 0],
       [3, 1],
       [4, 2],
       [5, 3],
       [6, 4],
       [7, 5],
       [3, 0],
       [4, 1],
       [5, 2],
       [6, 3],
       [7, 4],
       [4, 0],
       [5, 1],
       [6, 2],
       [7, 3],
       [5, 0],
       [6, 1],
       [7, 2],
       [6, 0],
       [7, 1],
       [7, 0]])

In [22]:
def gen_taus(no_data, d):
    rhos, cholesky_decompositions = gen_rhos_chos(no_data, d)

    dimension = 2**d
    tau_size = dimension**2

    all_taus = np.zeros((no_data, tau_size))
    indices = generate_index_pairs(d)

    for no in range(no_data):
        cholesky = cholesky_decompositions[no]
        extracted_values = np.zeros(tau_size)

        index=0
        for i in range(cholesky.shape[0]):
            extracted_values[i] = cholesky[i, i].real

        index = cholesky.shape[0]
        for i, j in indices:
            extracted_values[index] = cholesky[i, j].real
            extracted_values[index + 1] = cholesky[i, j].imag
            index += 2

        all_taus[no] = extracted_values

    return all_taus

In [24]:
tau1 = gen_taus(1,d)
tau1.shape

(1, 64)

In [34]:
def gen_nvals_taus_rhos(no_data, d):
    rhos, cholesky_decompositions = gen_rhos_chos(no_data, d)

    dimension = 2**d
    tau_size = dimension**2

    all_taus = np.zeros((no_data, tau_size))
    indices = generate_index_pairs(d)

    for no in range(no_data):
        cholesky = cholesky_decompositions[no]
        extracted_values = np.zeros(tau_size)

        index_tau=0
        for i in range(cholesky.shape[0]):
            extracted_values[i] = cholesky[i, i].real

        index_tau = cholesky.shape[0]
        for i, j in indices:
            extracted_values[index_tau] = cholesky[i, j].real
            extracted_values[index_tau + 1] = cholesky[i, j].imag
            index_tau += 2

        all_taus[no] = extracted_values

    nvals = np.zeros((no_data,6**d))

    for i1,rho in enumerate(rhos):
      for index,proj in enumerate(pauli_3d):
          nvals[i1,index] = np.trace(rho@proj)

    return nvals, all_taus, rhos

In [ ]:
tau_train = gen_taus(no_data, d)

In [ ]:
2**(2*d)

16

In [ ]:
tau_train[np.random.randint(no_data)]

array([ 7.47908073e-02,  6.91677910e-04,  1.91193049e-04,  3.99655479e-04,
        2.62928813e-01, -1.79774284e-01, -7.71304588e-05,  4.51239282e-04,
        2.04081742e-04, -1.43593635e-04,  9.28119769e-02,  2.02140389e-01,
        8.11682687e-04,  1.70703853e-03,  7.93698664e-01,  4.62081262e-01])

In [ ]:
tau_size = dimension**2
all_taus = np.zeros((no_data, tau_size))

primary_indices = [(1, 0), (2, 1), (3, 2)]
secondary_indices = [(2, 0), (3, 1), (3, 0)]

for no in range(no_data):
    cholesky = cholesky_decompositions[no]
    extracted_values = np.zeros(tau_size)

    for i in range(cholesky.shape[0]):
        extracted_values[i] = cholesky[i, i].real

    index = cholesky.shape[0]
    for i, j in ind:
        extracted_values[index] = cholesky[i, j].real
        extracted_values[index + 1] = cholesky[i, j].imag
        index += 2

    all_taus[no] = extracted_values


In [ ]:
# Generate rho matrices

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

tau_size = dimension**2
all_taus = np.zeros((no_data, tau_size))

primary_indices = [(1, 0), (2, 1), (3, 2)]
secondary_indices = [(2, 0), (3, 1), (3, 0)]

for no in range(no_data):
    cholesky = cholesky_decompositions[no]
    extracted_values = np.zeros(tau_size)

    for i in range(cholesky.shape[0]):
        extracted_values[i] = cholesky[i, i].real

    index = cholesky.shape[0]
    for i, j in ind:
        extracted_values[index] = cholesky[i, j].real
        extracted_values[index + 1] = cholesky[i, j].imag
        index += 2

    all_taus[no] = extracted_values


In [26]:
# Make 6^d projectors of Pauli {X,Y,Z} operators

paulis = [sigmax(),sigmay(),sigmaz()]
evectors = np.zeros((6,2,1),dtype=complex)

i=0
for pauli in paulis:
    vals, vecs = pauli.eigenstates()
    evectors[i] = vecs[0][:]
    evectors[i+1] = vecs[1][:]
    i += 2

projectors = np.zeros((6,2,2), dtype=complex)

for i, vectors in enumerate(evectors):
    projectors[i] = np.outer(vectors,vectors.conj())

In [27]:
pauli_2d = np.zeros((6**2,4,4), dtype=complex)

index=0
for i in projectors:
    for j in projectors:
        pauli_2d[index] = np.kron(i,j)
        index += 1

In [ ]:
np.kron(np.kron(projectors[0],projectors[0]),projectors[0]).shape

(8, 8)

In [28]:
pauli_3d = np.zeros((6**3,2**3,2**3), dtype=complex)

index=0
for i in pauli_2d:
    for j in projectors:
        pauli_3d[index] = np.kron(i,j)
        index += 1

In [ ]:
nvals = np.zeros((no_data,6**2))

for i1,rho in enumerate(rhos):
  for index,proj in enumerate(pauli_2d):
      nvals[i1,index] = np.trace(rho@proj)

<ipython-input-16-09091a42e478>:5: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [ ]:
# Generate rho matrices

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

tau_size = dimension**2
all_taus = np.zeros((no_data, tau_size))

primary_indices = [(1, 0), (2, 1), (3, 2)]
secondary_indices = [(2, 0), (3, 1), (3, 0)]

for no in range(no_data):
    cholesky = cholesky_decompositions[no]
    extracted_values = np.zeros(tau_size)

    for i in range(cholesky.shape[0]):
        extracted_values[i] = cholesky[i, i].real

    index = cholesky.shape[0]
    for i, j in ind:
        extracted_values[index] = cholesky[i, j].real
        extracted_values[index + 1] = cholesky[i, j].imag
        index += 2

    all_taus[no] = extracted_values

nvals = np.zeros((no_data,6**2))

for i1,rho in enumerate(rhos):
  for index,proj in enumerate(pauli_2d):
      nvals[i1,index] = np.trace(rho@proj)

In [49]:
dimension

8

In [50]:
def reconstruct_matrix_nn(taus):
    matrix = np.zeros((dimension, dimension), dtype=np.complex_)

    index = 0
    for i in range(dimension):
        matrix[i, i] = taus[index]
        index += 1

    indices = generate_index_pairs(d)

    for i, j in indices:
        real_part = taus[index]
        imag_part = taus[index + 1]
        matrix[i, j] = real_part + 1j * imag_part
        index += 2

    rho = matrix@(matrix.conj().T)
    rho = rho/np.trace(rho)

    return rho

In [51]:
def reconstruct_matrix_t(taus):
    matrix = np.zeros((dimension, dimension), dtype=np.complex_)

    index = 0
    for i in range(dimension):
        matrix[i, i] = taus[index]
        index += 1

    indices = generate_index_pairs(d)

    for i, j in indices:
        real_part = taus[index]
        imag_part = taus[index + 1]
        matrix[i, j] = real_part + 1j * imag_part
        index += 2

    rho = matrix@(matrix.conj().T)

    return rho

In [ ]:
((rho[0])@pauli_2d).shape

(36, 4)

In [ ]:
nvals = np.zeros((no_data,6**2))

for i1,rho in enumerate(rhos):
  for index,proj in enumerate(pauli_2d):
      nvals[i1,index] = np.trace(rho@proj)

<ipython-input-47-09091a42e478>:5: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [ ]:
# Generate rho matrices
no_data = 30000

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

tau_size = dimension**2
all_taus = np.zeros((no_data, tau_size))

for no in range(no_data):
    cholesky = cholesky_decompositions[no]
    extracted_values = np.zeros(tau_size)

    for i in range(cholesky.shape[0]):
        extracted_values[i] = cholesky[i, i].real

    index = cholesky.shape[0]
    for i, j in ind:
        extracted_values[index] = cholesky[i, j].real
        extracted_values[index + 1] = cholesky[i, j].imag
        index += 2

    all_taus[no] = extracted_values

nvals = np.zeros((no_data,6**2))

for i1,rho in enumerate(rhos):
  for index,proj in enumerate(pauli_2d):
      nvals[i1,index] = np.trace(rho@proj)

n_min = np.min(nvals)
n_max = np.max(nvals)

nvals = (nvals - np.min(nvals)) / (np.max(nvals) - np.min(nvals))

<ipython-input-35-dc5d83edfd35>:37: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [32]:
dense1 = 2500
dense2 = 1000

In [39]:
def build_cnn_model():
    model = Sequential()

    model.add(tf.keras.layers.Reshape((6, 36, 1), input_shape=(216,)))

    model.add(Conv2D(filters=25, kernel_size=(2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=25, kernel_size=(2, 2), strides=1, activation='relu'))

    model.add(Flatten())

    model.add(Dense(dense1, activation='relu'))

    model.add(Dense(dense2, activation='relu'))

    model.add(Dense(2**(2*d), activation='linear'))

    return model

model = build_cnn_model()

model.compile(
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.005),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape_1 (Reshape)                  │ (None, 6, 36, 1)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 5, 35, 25)           │             125 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 2, 17, 25)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 1, 16, 25)           │           2,525 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 400)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2500)                │       1,002,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1000)                │       2,501,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │          64,064 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,570,214 (13.62 MB)

 Trainable params: 3,570,214 (13.62 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# `x_train`, `y_train` are probabilities (n values) and labels (tau-vectors)
model.fit(nvals, all_taus, epochs=300, batch_size=100, validation_split=0.2)

Epoch 1/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0585 - mean_absolute_error: 0.1355 - val_loss: 0.0520 - val_mean_absolute_error: 0.1273
Epoch 2/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0513 - mean_absolute_error: 0.1258 - val_loss: 0.0498 - val_mean_absolute_error: 0.1230
Epoch 3/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0498 - mean_absolute_error: 0.1227 - val_loss: 0.0492 - val_mean_absolute_error: 0.1213
Epoch 4/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0493 - mean_absolute_error: 0.1212 - val_loss: 0.0491 - val_mean_absolute_error: 0.1206
Epoch 5/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0492 - mean_absolute_error: 0.1208 - val_loss: 0.0490 - val_mean_absolute_error: 0.1204
Epoch 6/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0492 - mean_absolute_error: 0.1207 - val_loss: 0.0489 - val_mean_absolute_error: 0.1203
Epoch 7/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0490 - mean_absolute_error: 0.1

In [41]:
from tensorflow.keras.callbacks import EarlyStopping

In [36]:
nvals[1000].shape

(216,)

In [35]:
nvals,all_taus,rhos = gen_nvals_taus_rhos(30000,d)

<ipython-input-34-f16c7555ff3e>:30: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [42]:
model1 = build_cnn_model()
model1.compile(
    optimizer='adam',
    loss='mean_absolute_error',
    metrics=['mean_squared_error']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history1 = model1.fit(
    nvals, all_taus,
    epochs=300,
    batch_size=100,
    validation_split=0.2
    )


Epoch 1/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0482 - mean_squared_error: 0.0143 - val_loss: 0.0431 - val_mean_squared_error: 0.0121
Epoch 2/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0413 - mean_squared_error: 0.0112 - val_loss: 0.0359 - val_mean_squared_error: 0.0087
Epoch 3/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0340 - mean_squared_error: 0.0080 - val_loss: 0.0287 - val_mean_squared_error: 0.0058
Epoch 4/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0264 - mean_squared_error: 0.0050 - val_loss: 0.0225 - val_mean_squared_error: 0.0037
Epoch 5/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0208 - mean_squared_error: 0.0033 - val_loss: 0.0197 - val_mean_squared_error: 0.0029
Epoch 6/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0185 - mean_squared_error: 0.0026 - val_loss: 0.0184 - val_mean_squared_error: 0.0026
Epoch 7/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0169 - mean_squared_error: 0.0022 - val_los

In [57]:
n_tests, tau_tests, rho_tests = gen_nvals_taus_rhos(500,d)

<ipython-input-34-f16c7555ff3e>:30: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [58]:
tau_nns = model1.predict(n_tests)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [59]:
fidelities = []
for nns, tests in zip(tau_nns,tau_tests):
  fidelities.append(fid_taus(tests,nns))

In [62]:
print(f'Average fidelity over {len(tau_nns)} test matrices is {np.mean(fidelities)*100:.2f} ± {np.std(fidelities)*100/np.sqrt(len(fidelities)):.2f} %')

Average fidelity over 500 test matrices is 94.11 ± 0.37 %


In [ ]:
model2 = build_cnn_model()
model2.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history2 = model2.fit(
    nvals2, all_taus2,
    epochs=300,
    batch_size=100,
    validation_split=0.2,
)

Epoch 1/300


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


240/240 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.0435 - mean_absolute_error: 0.1144 - val_loss: 0.0292 - val_mean_absolute_error: 0.0911
Epoch 2/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0279 - mean_absolute_error: 0.0880 - val_loss: 0.0237 - val_mean_absolute_error: 0.0804
Epoch 3/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0227 - mean_absolute_error: 0.0784 - val_loss: 0.0191 - val_mean_absolute_error: 0.0712
Epoch 4/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0186 - mean_absolute_error: 0.0704 - val_loss: 0.0162 - val_mean_absolute_error: 0.0655
Epoch 5/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0156 - mean_absolute_error: 0.0641 - val_loss: 0.0139 - val_mean_absolute_error: 0.0603
Epoch 6/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0132 - mean_absolute_error: 0.0587 - val_loss: 0.0124 - val_mean_absolute_error: 0.0564
Epoch 7/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0118 - mean_absolute_error: 0.0549 - val_lo

In [ ]:
n_t2, tau_t2, rho_t2 = gen_nvals_taus_rhos(1,2)

<ipython-input-169-dc7870c07db8>:30: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [ ]:
tau_nn2 = model2.predict(n_t2)

rho_nn2 = reconstruct_matrix(tau_nn2[0])

fid2 = fidelity(rho_t2[0], rho_nn2)
print(f'Fidelity of adam, MSE model: {100*fid2:.2f}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Fidelity of adam, MSE model: 99.94%


In [ ]:
print(np.min(nvals))
print(np.max(nvals))

0.0
1.0


In [ ]:
tau_nn = model.predict(nvals_t)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [ ]:
for i in range(len(tau_nn[0])):
    print(tau_nn[0,i],test_taus[0,i])

0.5371624 0.5257821498076084
0.0007096551 0.00016558421698149864
0.00016883017 0.000249638919107115
-0.0007570129 0.00018187496672064188
0.041995242 0.06895731200428411
-0.16272397 -0.14827102759802435
1.6191218e-05 -5.054292815108513e-05
0.0004988355 -3.2483250377041935e-05
-1.0312226e-05 8.596593542801998e-05
-0.0002111858 -6.833238988607688e-05
-0.57886195 -0.5684628810028554
0.38354665 0.35977967842484815
-0.0012810384 -4.391223745567451e-05
0.00030996412 4.423353698029809e-06
-0.15800397 -0.16243991687706466
0.47313604 0.466728391386248


In [ ]:
mse = 0
for i in range(len(test_taus[0])):
    mse += (test_taus[0,i]-tau_nn[0,i])**2

print(mse)

0.0018022209660357729


In [ ]:
print(no_data,d)

30000 2


In [ ]:
nvals,taus,rhos = gen_nvals_taus_rhos(no_data,d)

<ipython-input-9-dc7870c07db8>:30: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [ ]:
tau_tf = tf.convert_to_tensor(taus[5000])
tau_tf

<tf.Tensor: shape=(16,), dtype=float64, numpy=
array([ 1.38336767e-01,  5.98142315e-04,  1.78718113e-04,  2.67300196e-04,
       -3.68720976e-01,  3.44640887e-01, -2.70929251e-04, -1.37759379e-04,
        1.44805543e-05, -1.12627571e-04,  2.64897651e-01, -7.65324211e-02,
       -5.00571700e-04,  7.34407793e-04, -1.23188875e-01, -7.96822193e-01])>

In [ ]:
rho_tf = tf_reconstruct_t(tau_tf)
rho_tf

TypeError: Expected tensor 0.13833676743401946 with dtype tf.float32, but got dtype tf.float64.

In [ ]:
rho_nn

array([[ 0.27537991+0.j        ,  0.02152914+0.08342153j,
        -0.29675745-0.19662776j, -0.08100179-0.24255635j],
       [ 0.02152914-0.08342153j,  0.02695472+0.j        ,
        -0.08276537+0.07452483j, -0.07981178+0.00557486j],
       [-0.29675745+0.19662776j, -0.08276537-0.07452483j,
         0.46019164+0.j        ,  0.26048099+0.20354805j],
       [-0.08100179+0.24255635j, -0.07981178-0.00557486j,
         0.26048099-0.20354805j,  0.23747373+0.j        ]])

In [ ]:
test_rhos[0]

array([[ 0.27644687+6.04728294e-18j,  0.03625652+7.79582596e-02j,
        -0.29888764-1.89165733e-01j, -0.08540801-2.45397457e-01j],
       [ 0.03625652-7.79582596e-02j,  0.02673944-3.84512384e-19j,
        -0.09254458+5.94771414e-02j, -0.08040373-8.09920264e-03j],
       [-0.29888764+1.89165733e-01j, -0.09254458-5.94771414e-02j,
         0.45259153+2.07558995e-19j,  0.26026048+2.06875204e-01j],
       [-0.08540801+2.45397457e-01j, -0.08040373+8.09920264e-03j,
         0.26026048-2.06875204e-01j,  0.24422217+8.01907437e-19j]])

In [44]:
from scipy.linalg import sqrtm

In [45]:
def fidelity(rho_t, rho_nn):
    sqrt_nn = sqrtm(rho_nn)
    matrix = sqrt_nn@rho_t@sqrt_nn
    tr = np.trace(sqrtm(matrix))
    return np.abs(tr)**2

In [46]:
def fid_taus(tau_t, tau_nn):
    rho_t = reconstruct_matrix_t(tau_t)
    rho_nn = reconstruct_matrix_nn(tau_nn)
    return fidelity(rho_t, rho_nn)

In [ ]:
fidelity(test_rhos[0], rho_nn)

0.99948127099706151087

In [ ]:
def fidelity_metric(tau_ts, tau_nns):
    fidelities=np.zeros(len(tau_ts))
    tau_ts.numpy()
    tau_nns.numpy()
    for i in range(len(tau_ts)):
      tau_t = tau_ts[i]
      tau_nn = tau_nns[i]
      rho_t = reconstruct_matrix_t(tau_t)
      rho_nn = reconstruct_matrix_nn(tau_nn)
      fidelity_value = fidelity(rho_t, rho_nn)
      fidelities[i] = fidelity_value
    return mean(fidelities)